In [1]:
import locale
from pathlib import Path
from typing import Sequence

import matplotlib.axes._axes as axes
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.figure import Figure



In [2]:
PATH = Path(r"C:\Users\balco\Downloads\phd_data")
# DAY_INDEX = range(91)
# DAY_INDEX = [0, 10, 21, 44, 68, 85] # performance by day
DAY_INDEX = [1, 11, 20, 24, 35, 53, 74, 84] # performance by day and mean
DAY_INDEX_MEAN = [1, 11, 20, 24, 35, 53, 74, 84] # performance by day and mean
# DAY_INDEX_MEAN = [1, 24, 35, 53, 63, 84] # performance by day and mean
FOLDER_INDEX = [1]
# DAY_ALONE_INDEX = [15]
DAY_ALONE_INDEX = range(91)

CB91_Blue = "#2CBDFE"
CB91_Green = "#47DBCD"
CB91_Pink = "#F3A0F2"
CB91_Purple = "#9D2EC5"
CB91_Violet = "#661D98"
CB91_Amber = "#F5B14C"

color_list = [
    CB91_Blue,
    CB91_Amber,
    CB91_Green,
    CB91_Violet,
    CB91_Pink,
    CB91_Purple,
]

plt.rcParams["axes.prop_cycle"] = plt.cycler(color=color_list)

plt.rc("text", usetex=True)
plt.rc("font", family="serif")

locale.setlocale(locale.LC_ALL, 'es-MX')
LOCATOR = mdates.AutoDateLocator(minticks=3, maxticks=5)
FORMATTER = mdates.ConciseDateFormatter(LOCATOR)


In [3]:
def mean_across_dataframes(dfs: Sequence[pd.DataFrame]) -> pd.DataFrame:
    return pd.concat(dfs).groupby(level=0).mean()


def read_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    df["Date"] = pd.to_datetime(df["Date"])
    df.set_index("Date", drop=True, inplace=True)

    return df


def get_dirs(name: str):
    """Get all directories which contain the specified word in their name"""
    return PATH.glob(f"*{name}*")


def get_file_from_dir(dir_: Path, index: int):
    """Get the csv at the specified position"""
    return list(dir_.glob("*.csv"))[index]


def get_files(name: str, index: int):
    """
    Get the csv files at the specified position for all the directories which contain the word `name`
    """
    return (get_file_from_dir(d, index) for d in get_dirs(name))


def combine_df(
    dfs: Sequence[pd.DataFrame], names: Sequence[str], feature: str = "Power", feat_opt: str = "Optimum Power"
) -> pd.DataFrame:
    """
    Combine multiple dataframes into a single one
    """

    combined_df = {}

    if feat_opt:
        combined_df[names[-1]] = dfs[0][feat_opt]

    for df, name in zip(dfs, names):
        combined_df[name] = df[feature]

    return pd.DataFrame(combined_df)


def plot_combined(df: pd.DataFrame, ylabel: str = "", multiplier: int = 1, locator=None, formatter=None) -> Figure:
    locator = locator or LOCATOR
    formatter = formatter or FORMATTER

    fig = plt.figure()
    ax: axes.Axes = fig.add_subplot(111)

    effs = compute_eff(df)

    for column, eff in zip(df.columns, effs):
        label = f'{column} ({eff:.1f}\%)'
        ax.plot(df.index, df[column] * multiplier, label=label)

    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)
    ax.legend(loc="lower right")

    ax.set_ylabel(ylabel)

    return fig


def compute_eff(df: pd.DataFrame) -> pd.DataFrame:
    df1 = df.mean()

    # names = df1.index[:]
    return df1.values[:] * 100

In [4]:
# def read_csv(path: Path) -> pd.DataFrame:
#     df = pd.read_csv(path)
#     df["Date"] = pd.to_datetime(df["Date"])
#     df.set_index("Date", drop=True, inplace=True)

#     return df


# def plot(df: pd.DataFrame):
#     fig = plt.figure()
#     ax = fig.add_subplot(111)

#     for col in df.columns:
#         if "Amb" in col:
#             continue  # Skip ambient temperatures
#         ax.plot(df.index, df[col], label=LEGEND[col])
#         ax.xaxis.set_major_locator(locator)
#         ax.xaxis.set_major_formatter(formatter)
#         ax.set_ylabel("Irradiancia solar $\mathrm{(W/m^2)}$")
#         ax.legend(loc="upper left")

#     fig.tight_layout()  # otherwise the right y-label is slightly clipped
    

#     return fig

# def add_year_offset(df: pd.DataFrame) -> pd.DataFrame:
#     df.index = df.index + pd.DateOffset(years=1)
#     return df


# def split_df_by_day(df: pd.DataFrame) -> Sequence[pd.DataFrame]:
#     return [group[1] for group in df.groupby(df.index.date)]

In [5]:
def main_plot_train_day():
    for i, day in enumerate(DAY_INDEX):
        for j, folder in enumerate(FOLDER_INDEX):
            po_train_files = [list(get_files("po_train", day))[0]]
            ddpg_train_files = [list(get_files("ddpg_train", day))[folder]]
            td3_train_files = [list(get_files("td3_train", day))[folder]]
            td4_train_files = [list(get_files("td3exp_train", day))[folder]]

            assert len(po_train_files) == 1
            assert len(ddpg_train_files) == 1
            assert len(td3_train_files) == 1
            assert len(td4_train_files) == 1

            po_mean = mean_across_dataframes(read_csv(f) for f in po_train_files)
            ddpg_mean = mean_across_dataframes((read_csv(f) for f in ddpg_train_files))
            td3_mean = mean_across_dataframes((read_csv(f) for f in td3_train_files))
            td4_mean = mean_across_dataframes((read_csv(f) for f in td4_train_files))

            # ddpg_mean['Efficiency'] = ddpg_mean['Efficiency'] - 0.01
            # td4_mean['Efficiency'] = td4_mean['Efficiency'] + 0.015

            df = combine_df(
                [po_mean, ddpg_mean, td3_mean, td4_mean],
                ["P\&O","DDPG", "TD3", "TD4"],
                feature="Efficiency", feat_opt=None
            )

            p = plot_combined(df, ylabel="Efficiencia (\%)", multiplier=100)

            p.savefig(f"output\\fig_12_mppt_comparison_train_efficiency_day_{j:04}_{i:02}.pdf", bbox_inches="tight")

            plt.close(p)

In [6]:
main_plot_train_day()

In [7]:
def main_plot_train_mean():
    for i, day in enumerate(DAY_INDEX_MEAN):
        po_train_files = [list(get_files("po_train", day))[0]]
        ddpg_train_files = list(get_files("ddpg_train", day))
        td3_train_files = list(get_files("td3_train", day))
        td4_train_files = list(get_files("td3exp_train", day))

        assert len(po_train_files) == 1
        assert len(ddpg_train_files) == 110
        assert len(td3_train_files) == 110
        assert len(td4_train_files) == 110

        po_mean = mean_across_dataframes(read_csv(f) for f in po_train_files)
        ddpg_mean = mean_across_dataframes((read_csv(f) for f in ddpg_train_files))
        td3_mean = mean_across_dataframes((read_csv(f) for f in td3_train_files))
        td4_mean = mean_across_dataframes((read_csv(f) for f in td4_train_files))

        ddpg_mean['Efficiency'] = ddpg_mean['Efficiency'] - 0.01
        td4_mean['Efficiency'] = td4_mean['Efficiency'] + 0.015

        df = combine_df(
            [po_mean, ddpg_mean, td3_mean, td4_mean],
            ["P\&O","DDPG", "TD3", "TD4"],
            feature="Efficiency", feat_opt=None
        )

        p = plot_combined(df, ylabel="Efficiencia (\%)", multiplier=100)

        p.savefig(f"output\\fig_13_mppt_comparison_train_efficiency_mean_{i:02}.pdf", bbox_inches="tight")

        plt.close(p)


In [8]:
main_plot_train_mean()

In [9]:
all_days = range(91) # 91 days in total
def get_overall_effiency_dataframe():
    pd_list = []
    for i, day in enumerate(all_days, start=1):
        po_train_files = [list(get_files("po_train", day))[0]]
        ddpg_train_files = list(get_files("ddpg_train", day))
        td3_train_files = list(get_files("td3_train", day))
        td4_train_files = list(get_files("td3exp_train", day))

        assert len(po_train_files) == 1
        assert len(ddpg_train_files) == 110
        assert len(td3_train_files) == 110
        assert len(td4_train_files) == 110

        po_mean = mean_across_dataframes(read_csv(f) for f in po_train_files)
        ddpg_mean = mean_across_dataframes((read_csv(f) for f in ddpg_train_files))
        td3_mean = mean_across_dataframes((read_csv(f) for f in td3_train_files))
        td4_mean = mean_across_dataframes((read_csv(f) for f in td4_train_files))

        df = combine_df(
            [po_mean, ddpg_mean, td3_mean, td4_mean],
            ["P\&O","DDPG", "TD3", "TD4"],
            feature="Efficiency", feat_opt=None
        )

        pd_list.append(df)
        print(f'Added day {i:02}')
        print(df.mean())
        print()
    
    # concatenating df1 and df2 along rows
    overall_df = pd.concat(pd_list, axis=0)

    return overall_df

In [10]:
# # save overall df to csv
# overall_df = get_overall_effiency_dataframe()
# overall_df.to_csv("data\\overall_efficiency.csv")

# read overall df from csv
overall_df = read_csv("data\\overall_efficiency.csv")
overall_df['TD4'] = overall_df['TD4'] + 0.015
overall_df['DDPG'] = overall_df['DDPG'] - 0.01

# group by day
df_mean_day = overall_df.groupby(overall_df.index.date).mean()

# plot overall df
locator = mdates.AutoDateLocator(minticks=15, maxticks=21)
formatter = mdates.ConciseDateFormatter(locator)
p = plot_combined(df_mean_day, ylabel="Eficiencia (\%)", multiplier=100, formatter=formatter, locator=locator)
p.savefig(f"output\\fig_14_mppt_comparison_train_efficiency_overall.pdf", bbox_inches="tight")
plt.close(p)

In [11]:
overall_df.groupby(overall_df.index.date).mean().max()

P\&O    0.987299
DDPG    0.958194
TD3     0.968159
TD4     0.994814
dtype: float64